# Wiederholung

* Konzepte einfacher Lernverfahren kennengelernt und __implementiert__ 
    * Perzeptron, Adaline, ...
    * immer mit der gleichen Struktur 
        * ```init, net-input, activation, predict, fit [,partial_fit]```
    
* Standardisierte Merkmale besprochen

* Gradientenabstiegsverfahren 




# Klassifizierung mit scikit-learn 

* weitere stabile und verbreitete Lernalgorithmen kennenlernen
    * logistische Regression, Support Vector Machine, Entscheidungsbäume
* Unterschiede zwischen überwachten Lernalgorithmen kennenlernen
    * Stärken und Schwächen von Klassifizierern 
* Kennenlernen der scikit-learn Bibliothek



## Auswahl der Klassifizierungsalgorithmen 

"No Free Lunch" 
* _Es gibt keinen Klassifizierer, der für alle denkbaren Szenarien geeignet ist_ 
* Jeder Klassifizierer hat gewisse Eigenschaften und geht von bestimmten Annahmen aus
* https://ieeexplore.ieee.org/document/6795940

Pragmatischer Ansatz
* verschiedene Lernalgorithmen miteinander vergleichen
    * das beste Modell für eine Aufgabe verwenden
* Auswahl hängt u.a. ab von
    * Anzahl der Merkmale
    * "Rauschen" in der Datensammlung
    * der linearen Trennbarkeit der Klassen
    * ... 



__*Daten sind das wichtigste!*__
* Vorhersagekraft eines Algorithmus hängt von den zum Lernen verfügbaren Daten ab! 
* Bedarf an Rechenleistung hängt von den zum Lernen verfügbaren Daten ab 

Daten sind alles! 

Allgemeines Vorgehen
1. Auswahl der Merkmale und Sammeln von Trainingsdaten 
1. Kriterien für Leistungsbewertung festlegen 
1. Auswahl eines Klassifizierers und eines Optimierungsalgorithmus
1. Bewertung der Leistung des Modells
1. Feinabstimmung des Algorithmus



# Prezeptron mit scikit-learn 

Nochmals an der Iris-Datensammlung
* wieder mit nur zwei Merkmalen der Sammlung (Länge und Breite der Blütenblätter)

In [ ]:
from sklearn import datasets
import numpy as np

iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target

print('Klassenbezeichner:', np.unique(y))

* Verwendet ganzahlige Klassenbezeichungen 
    * unique() wandelt Text in ganzahlige Werte um 
    * Performanz bei Ganzzahlen ist besser als bei Strings



### Erinnerung: Typisches System für Machine Learning

* Typischer Ablauf 

<img src="./Bilder/BuchKap01/01_09.png" style="height:50%" />



### Teilen der Datenmenge
* in eine __Trainingsdatenmenge__ und in eine __Testdatenmenge__
    * hier 30 % Testdaten 
    + train_test_split durchmischt die Daten auch
        * siehe Dokumentation: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

print('Labels counts in y:', np.bincount(y))
print('Labels counts in y_train:', np.bincount(y_train))
print('Labels counts in y_test:', np.bincount(y_test))

### Standardisierung der Merkmale

<img src="./Bilder/BuchKap02/02_13.png" 
width=30% align=right float=left/>

* Erinnerung: 
    * verschieben der Merkmale auf den Mittelwert 0 
    * skalieren der Merkmale, sodass die Standardabweichung 1 ist


$$\\[5mm] x_j^{\prime} = \frac{x_j - \mu_j}{\sigma_j}
$$

* StandardScaler ist Teil des scikit-learn Preproccessing Moduls
    * siehe Dokumentation: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

* Beide Datensätze (Training und Test) müssen gleich transferiert werden 



### Auswahl und Anwenden des Algorithmus - hier Prezeptron 

* Teil von scikit-learn 
* sehr ähnlich der bisher verwendeten Klasse
    * eta0: Lernrate
        * auch hier muss ausprobiert werden
    * n_iter: Epochen
    * fit(): Trainieren des Modells (Bestimmung der Gewichte)

In [ ]:
from sklearn.linear_model import Perceptron

ppn = Perceptron(max_iter=40, eta0=0.1, random_state=2)
ppn.fit(X_train_std, y_train)

### Bewertung der Ergebnisse

* Anwenden der predict() Methode auf die Testdaten 
* Anzahl der Fehlklassifizierungen ausgeben 
    * besser: Korrektheitsklassifizierungsrate (Accuracy) = 1 - Fehlklassifizierungsrate
        * beachte: Jeder Klassifizierer hat eine Funktion score mit der dieser Wert bestimmt werden kann
        * beachte: das metrics Modul von scikit-learn liefert auch Bewertungskriterien

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = ppn.predict(X_test_std)
print('Fehlkassifizierungen: %d' % (y_test != y_pred).sum())

print('Korrektheitsklassifizierungsrate: %.2f' % accuracy_score(y_test, y_pred))
print('Korrektheitsklassifizierungsrate: %.2f' % ppn.score(X_test_std, y_test))

## Retrospektive: ML-Prozess

<img src="./Bilder/BuchKap01/01_09.png" style="width:30%; align:right; float:right"/>

* die Daten wurden unterteilt
* die Daten wurden skaliert
* das Modell/der Klassifizierungsalgorithmus wurde ausgewählt
    * mit den Trainingsdaten optimiert
* das Modell wurde bewertet
    * mit den Testdaten überprüft 


Beachte: 
* Durch die Trennung von Trainings- und Testdaten: 
    * **Überanpassung** kann gefährlich werden! 
    * Das Modell kann zwar die Muster in den Trainingsdaten gut erkennen, aber nicht auf unbekannte Daten (z.B. die Testdaten) verallgemeinern!   



### Visualisierung

Anpassung der plot_decision_region

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl, 
                    edgecolor='black')

    # highlight test samples
    if test_idx:
        # plot all samples
        X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    c='',
                    edgecolor='black',
                    alpha=1.0,
                    linewidth=1,
                    marker='o',
                    s=100, 
                    label='test set')

Ausgabe des Modells

In [ ]:
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))

plot_decision_regions(X=X_combined_std, y=y_combined,
                      classifier=ppn, test_idx=range(105, 150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

Diskussion: 
* die drei Blumenarten sind nicht vollständig voneinander durch eine lineare Entscheidungsgrenze trennbar
* Perzeptron konvergiert nicht, wenn Datenmengen nicht vollkommen linear trennbar sind
    * Weil es in jeder Epoche mindestens ein fehlklassifiziertes Exemplar gibt, werden die Gewichtungen in jeder Epoche aktualisiert 
* von der Verwendung des Perzeptrons ist im Allgemeinen abzuraten 
    * hier nur als Einstieg in scikit-learn verwendet
* Weitere Information
    * https://scikit-learn.org/stable/

In [ ]:
help(Perceptron)

# Logistische Regression 

**Wichtig** Logistische Regression ist ein Klassifizierungsmodell
* und kein Verfahren im Zusammenhang mit stetiger Regression!



## Grundlagen 

* basiert auf Wahrscheinlichkeitsmodell 
    * $p$ = Wahrscheinlichkeit für das Eintreten eines bestimmten Ereignisses
        * "Positivereignis" 
        * z.B. die Wahrscheinlichkeit, dass ein Patient an einer Krankheit leidet
    * Chancenverhältnis für das Ereignis 

$$\frac{p}{1-p}$$

* Definition der logit-Funktion

$$logit(p) = log (\frac{p}{1-p})$$

* Eigenschaften 
    * log ist natürlicher Logarithmus
    * logit nimmt Werte zwischen 0 und 1 entgegen
        * bildet diese auf den gesamten Bereich der reellen Zahlen ab



In [ ]:
import numpy as np

# Werte-Listen für Sinus-Funktion generieren:
x = np.arange(0.000001, 0.999999, 0.000001)  # Start, Stop, Step
y = np.log(x/(1-x))

# Sinus-Kurve plotten:
plt.plot(x, y)
plt.axis( [-0.1, 1.1, -10, 10] )
plt.grid(True)
plt.show() 

* dem Positivereignis sei die Klassenbezeichnung $y=1$ zugeordnet
    * $p(y=1 | x )$ bedingte Wahrscheinlichkeit, dass ein bestimmtes Objekt bei gegebenem Merkmal $x$ zur Klasse 1 gehört
    * formuliere lineare Beziehungen zwischen Merkmalen und dem Logarithmus des Chancenverhältnisses
        * Beachte: $w_0$ ist Bias und $x_0 = 1$

$$logit(p(y=1 | x )) = w_0 x_0 + w_1 x_1 + ... + w_m x_m = \sum_{i=1}^{m} w_i x_i = w^T x = z 
$$

was wir eigentlich wollen
* Wahrscheinlichkeit das ein Objekt zu einer bestimmten Klasse gehört
* Kehrwert der $logit()$ Funktion 
* Logistische Funktion 

$$\phi(z) = \phi (w^T x) = \frac{1}{1 + e^{-z}} = \frac{1}{1 + e^{- w^T x}}
$$



### Sigmoidfunktion

In [ ]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

z = np.arange(-7, 7, 0.1)
phi_z = sigmoid(z)

plt.plot(z, phi_z)
plt.axvline(0.0, color='k')
plt.ylim(-0.1, 1.1)
plt.xlabel('z')
plt.ylabel('$\phi (z)$')

# y axis ticks and gridline
plt.yticks([0.0, 0.5, 1.0])
ax = plt.gca()
ax.yaxis.grid(True)

plt.tight_layout()
plt.show()

Eigenschaften der Sigmoid-Funktion 

* $z \rightarrow \infty$; dann  $e^{-z}$ wird klein; dann geht $\phi(z)$ gegen 1   
* $z \rightarrow -\infty$; dann  $e^{-z}$ wird sehr groß; dann geht $\phi(z)$ gegen 0
* Reelle Zahlen werden auf das Intervall $[0,1]$ abgebildet
* $z=0$; dann $e^{-z}$ wird 1; dann ist $\phi(z) = 0.5$



### Vergleich Logistische Regression mit Adaline

* Adaline: Aktivierungsfunktion durch lineare Abbildung $\phi(z) = z$
* Jetzt: Aktivierungsfunktion durch Sigmoid

<img src="./Bilder/Buchkap03/03_03.png" style="height:50%" />



#### Interpretation der logistischen Regression
* Ausgabe der Sigmoid $\phi(z) = P(y=1 | x;w)$ wird als Wahrscheinlichkeit interpretiert, dass ein bestimmtes Objekt bei gegebenen Gewichtungen $w$ und Merkmalen $x$ zur Klasse 1 gehört 
* Bsp: 
    * eine Blume ergibt $\phi(z) = 0.8$, dann ist die Wahrscheinlichkeit  eine versicolor zu sein 80%
    * die Wahrscheinlichkeit, dass es sich um eine setosa handelt ist dann 20%
$$P(y=0  |x;w) = 1 - P(y=1 | x; w) = 0.2
$$



#### Schwellwertfunktion 

Binäre Entscheidung nach Aktivierungsfunktion mit einer Schwellenfunktion notwendig

$$
\hat y =\begin{cases}
 1 & \text{wenn }  \phi(z) \ge 0.5 \\[4mm]
0 & \text{andernfalls}
\end{cases}
$$

äquivalent zu (siehe Sigmoidfunktion)  

$$
\hat y =\begin{cases}
 1 & \text{wenn }  z \ge 0 \\[4mm]
0 & \text{andernfalls}
\end{cases}
$$



#### Vorteile der Wahrscheinlichkeiten 
* Anwendungsfälle, die nicht nur klassifizieren, sondern auch die Wahrscheinlichkeiten angeben
    * Bsp: Wettervorhersage
        * Es wird morgen regnen (Klassifikation)
        * mit der Wahrscheinlichkeit von 40% (Wiederverwendung der Wahrscheinlichkeiten)
    * Bsp: Medizin
        * Logistische Regression ist sehr verbreitet
        * Ein Patient mit gegebenen Symptomen leidet mit einer bestimmten Wahrscheinlichkeit an einer bestimmten Krankheit



#### Ermittlung der Gewichte

Bisher: Straffunktion minimieren um Gewichte $w$ zu ermitteln 

$$J(w) =  \frac{1}{2} \sum_i (\phi(z^{(i)}) - y^{(i)})^2
$$

Jetzt: Wie sieht die Straffunktion bei der logistischen Regression aus? 



* Definition Likelihood $L$
    * Annahme: die einzelnen Objekte sind voneinander unabhängig
    * Maximierung der Funktion notwendig

$$L(w) = P (y  | x; w) = \prod_{i=1}^n P(y^{(i)} | x^{i}; w) 
= \prod_{i=1}^n \left( \phi(z^{(i)}) \right) ^{y^{(i)}} \left( 1- \phi(z^{(i)}) \right)^{1-y^{(i)}}
$$

* Definition: Log-Likelihood
    * in der Praxis besser einsetzbar
        * kein arithmetrischer Unterlauf, wenn Wahrscheinlichkeiten sehr klein sind
    * Produkt der Faktoren kann als Summe der Faktoren geschrieben werden
 
$$l(w) = log (L(w)) = \sum_{i=1}^{n} y^{(i)} \log (\phi(z^{(i)})
+ (1 - y^{(i)}) \log (1 - \phi(z^{(i)}))$$



* Definition einer Straffunktion 
    * um Gradientenabstiegsverfahren wiederverwenden zu können

$$J(w) = - l(w) = \sum_{i=0}^{n} - y^{(i)} \log (\phi(z^{(i)}) 
- (1 - y^{(i)}) \log (1 - \phi(z^{(i)})$$

Beispiel für Straffunktion: 
* Wert für ein einzelnes Objekt

$$ J(\phi(z), y, w) = - y \log (\phi(z)) - (1 - y) \log (\phi(z)) 
$$

 * der erste oder zweite Term wird Null, wenn y=0 oder y=1 ist

$$
J(\phi(z), y, w) = \begin{cases}
 - \log(\phi(z)) & \text{wenn }  y = 1 \\[4mm]
 - \log (1 - \phi(z)),  & y = 0 
\end{cases}
$$



Darstellung der Straffunktionen

In [ ]:

def cost_1(z):
    return - np.log(sigmoid(z))

def cost_0(z):
    return - np.log(1 - sigmoid(z))

z = np.arange(-10, 10, 0.1)
phi_z = sigmoid(z)

c1 = [cost_1(x) for x in z]
plt.plot(phi_z, c1, label='J(w) if y=1')

c0 = [cost_0(x) for x in z]
plt.plot(phi_z, c0, linestyle='--', label='J(w) if y=0')

plt.ylim(0.0, 5.1)
plt.xlim([0, 1])
plt.xlabel('$\phi$(z)')
plt.ylabel('J(w)')
plt.legend(loc='best')
plt.tight_layout()
plt.show() 

* Straffunktion = 0, wenn richtig vorhergesagt wird 
* falsche Vorhersagen werden zunehmend härter bestraft
    * ist die Vorhersage ganz falsch, geht die Straffunktion gegen unendlich




### Gradientenabstiegsverfahren bei logistischer Regression 

Betrachte Ableitungen der Log-Likelihood Funktion 

$$\frac{\partial}{\partial w_j} l(w) \\ 
= \frac{\partial}{\partial w_j} \sum_{i=0}^{n} y^{(i)} \log (\phi(z^{(i)}) + (1 - y^{(i)}) \log (1 - \phi(z^{(i)})) \\
= \left(y \frac{1}{\phi(z)} - (1-y) (\frac{1}{1-\phi(z))}\right) \frac{\partial}{\partial w_j} \phi(z)
$$ 



Betrachte partielle Ableitung der Sigmoid-Funktion nach z: 

$$\frac{\partial}{\partial z} \phi(z)\\[4mm]
= \frac{\partial}{\partial z} \frac{1}{1 + e^{-z}} \\[4mm]
= \frac{1}{(1 + e^{-z})^2} \cdot e^{-z} \\[4mm]
= \frac{1}{(1 + e^{-z})} \frac{e^{-z}}{( 1 + e^{-z})} = \frac{1}{(1 + e^{-z})} \frac{(1 + e^{-z} - 1)}{(1 + e^{-z})} \\[4mm]
= \frac{1}{(1 + e^{-z})}  \cdot (1 - \frac{1}{1 + e^{-z}}) \\[4mm]
= \phi(z) ( 1 - \phi(z))$$



Substituiere in der Ausgangsgleichung der Log-Likelihood : 

$$\left(y \frac{1}{\phi(z)} - (1-y) (\frac{1}{1-\phi(z))}\right) \frac{\partial}{\partial w_j} \phi(z) \\[4mm]
= \left(y \frac{1}{\phi(z)} - (1-y) (\frac{1}{1-\phi(z))}\right) (\phi(z) (1 - \phi(z)) \cdot \frac{\partial}{\partial w_j} z \\[4mm]
= \left(y (1 - \phi(z)) - (1-y) \phi(z) \right) \cdot \frac{\partial}{\partial w_j} z \\[4mm]
= \left(y (1 - \phi(z)) - (1-y) \phi(z) \right) \cdot x_j \\[4mm]
\left( y - \phi(z) \right) x_j $$



Aktualisierung der Gewichte

$$w_j := w_j + \eta \sum_{i=1}^n \left( y^{} - \phi(z^{(i)})\right) x^{(i)}$$

* als Vektoren 

$$w := w + \Delta w \\[4mm]$$

$$ \Delta w = \eta \nabla l(w)$$  

Statt Maximierung der Log-Likelihood auch Minimierung der Straffunktion $J$

$$w := w + \Delta w  = w - \eta \nabla J(w)$$  

**Gleiche Regel wie bei Adaline!**



### Implementierung einer Klasse für logistische Regression

Vorgehen:
1. verwende die Adaline-Implementierung
1. die Aktivierungsfunktion durch Sigmoidfunktion ersetzen
1. die Schwellwertfunktion ersetzen 
    * Rückgabewerte nun 0 oder 1 statt -1 oder 1
2. ersetze die Straffunktion mit 

$$
J(w) = \sum_i y^{(i)} log (\phi(z^{(i)})) + (1 - y^{(i)}) log (1 - \phi(z^{(i)}))
$$

* Anwendung auf eine binäre Klassifizierung möglich



### Implementierung logistische Regression 

* Beachte: Anwendung auf binäre Klassifizierungsaufgabe
    * betrachte nur Iris setosa und Iris versicolor

In [ ]:
class LogisticRegressionGD(object):
    """Logistic Regression Classifier using gradient descent.

    Parameters
    ------------
    eta : float
      Learning rate (between 0.0 and 1.0)
    n_iter : int
      Passes over the training dataset.
    random_state : int
      Random number generator seed for random weight
      initialization.


    Attributes
    -----------
    w_ : 1d-array
      Weights after fitting.
    cost_ : list
      Logistic cost function value in each epoch.

    """
    def __init__(self, eta=0.05, n_iter=100, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """ Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
          Training vectors, where n_samples is the number of samples and
          n_features is the number of features.
        y : array-like, shape = [n_samples]
          Target values.

        Returns
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.cost_ = []

        for i in range(self.n_iter):
            net_input = self.net_input(X)
            output = self.activation(net_input)
            errors = (y - output)
            self.w_[1:] += self.eta * X.T.dot(errors)
            self.w_[0] += self.eta * errors.sum()
            
            # note that we compute the logistic `cost` now
            # instead of the sum of squared errors cost
            cost = -y.dot(np.log(output)) - ((1 - y).dot(np.log(1 - output)))
            self.cost_.append(cost)
        return self
    
    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def activation(self, z):
        """Compute logistic sigmoid activation"""
        return 1. / (1. + np.exp(-np.clip(z, -250, 250)))

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, 0)
        # equivalent to:
        # return np.where(self.activation(self.net_input(X)) >= 0.5, 1, 0)


Ausführen des binären Problems

In [ ]:
X_train_01_subset = X_train[(y_train == 0) | (y_train == 1)]
y_train_01_subset = y_train[(y_train == 0) | (y_train == 1)]

lrgd = LogisticRegressionGD(eta=0.05, n_iter=1000, random_state=1)
lrgd.fit(X_train_01_subset,
         y_train_01_subset)

plot_decision_regions(X=X_train_01_subset, 
                      y=y_train_01_subset,
                      classifier=lrgd)

plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

#### Retrospektive

* Konzept der logistischen Regression besprochen 
    * an einfacher binären Klassifizierung
* eigene Implementierungen 


Jetzt: 

* Verwende scikit-learn 



### Logistische Regression mit scikit-learn

* hochoptimiert
* mehrere Klassen werden unterstützt
* Klasse sklearn.linear-model.LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100.0, random_state=1)
lr.fit(X_train_std, y_train)

plot_decision_regions(X_combined_std, y_combined,
                      classifier=lr, test_idx=range(105, 150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

* Bedeutung von C? 
    * wird später diskutiert! 

* Bestimmung der Wahrscheinlichkeiten von Objekten 
    * verwende die Funktion predict_proba
    * liefert für ein Objekt die Wahrscheinlichkeit, zu einer Klasse zu gehören

In [ ]:
lr.predict_proba(X_test_std[:3, :])

* Summe aller Wahrscheinlichkeiten für ein Objekt muss eins sein

In [ ]:
lr.predict_proba(X_test_std[:3, :]).sum(axis=1)

* Beste Vorhersage ist der Wert mit der größten Wahrscheinlichkeit

In [ ]:
lr.predict_proba(X_test_std[:3, :]).argmax(axis=1)

* Bestimmung der Klasse mit predict

In [ ]:
lr.predict(X_test_std[:3, :])

* Achtung: predict erwartet ein zweidimensionales Array
    * wenn nur ein Objekt ermittelt werden soll, dann muss dieses mit reshape in der richtige Format konvertiert werden

In [ ]:
lr.predict(X_test_std[0, :].reshape(1, -1))

## Überanpassung

* Häufiges Problem beim Machine Learning

* Modell funktioniert gut mit Trainingsdaten, aber mit unbekannten Daten (Testdaten) kommt es schlecht zurecht
    * Grund: Verallgemeinerung von Trainingsdaten ist nicht gelungen

* Überpassung == große Varianz
    * viele Parameter führen zu einem komplexen Modell

* Unteranpassung == großer Bias
    * nicht komplex genug, um die Muster in den Daten zu erkennen
    * funktioniert bei unbekannten Daten dann auch schlecht 



![UeberAnpassung](./Bilder/BuchKap03/03_07.png)



Begriffe
* Varianz: Maß für die **Konsistenz** der Modellvorhersage
    * kleine Varianz: ein bestimmtes Objekt wird immer gleich vorhergesagt, auch wenn man das Modell mit unterschiedlichen Trainingsdaten optimiert
    * Veränderlichkeit
* Bias: Maß für den systemeatischen Fehler, der nicht durch Zufälligkeiten geschuldet ist
    * wie stark weichen die Vorhersagen ab, wenn das Modell mehrfach mit unterschiedlichen Trainingsdaten optiminert wurde 

**Ziel: guter Bias-Varianz-Kompromis**



### Regularisierung

Methode um
* hohe Korrelationen zwischen Merkmalen (Kolinearität) zu handhaben
* Rauschen aus den Daten herauszufiltern 
* Überanpassung zu verhindern 

Idee: 
* zusätzliche Information (Bias) einführen, die extreme Parameter (Gewichte) bestrafen 



**L2-Regularisierung**

* weiterer Term in Straffunktion 

$$\frac{\lambda}{2} ||w||^2 = \frac{\lambda}{2} \sum_{j=1}^{m} w_j^2 
$$

* $\lambda$ Regularisierungsparameter

* Straffunktion logistische Regression mit L2-Regularisierung

$$j(w) = \left[ \sum_{i=1}^{n}  y^{(i)} \left( -\log( \phi(z^{(i)}))  \right) + \left ( 1 - y^{(i)}\right) \left( -\log(1 - \phi(z^{(i)}))\right)
\right ] + \left[ \frac{\lambda}{2} \sum_{j=1}^{m} w_j^2  \right ]  
$$



* Regularisierungsparameter $\lambda$ steuert die genaue Anpassung an die Trainingsdaten
    * $\lambda = 0$: strenge Anpassung 
    * $\lambda$ groß bedeutet stärkere Regularierung

* Parameter C in der Klasse LogisticRegression
    * Kehrwert der Regularisierung
    * bekommt bei Support-Vector-Machines noch mehr Bedeutung

$$ C = \frac{1}{\lambda}
$$ 

* kleines C: --> starke Regularisierung



Studie: Mehrere Trainingsläufe mit verschiedenen Regularisierungsparametern 
* zehn Durchläufe
* Darstellung des Gewichtungskoeffizienten

In [ ]:
weights, params = [], []
for c in np.arange(-5, 5):
    lr = LogisticRegression(C=10.**c, random_state=1)
    lr.fit(X_train_std, y_train)
    weights.append(lr.coef_[1])
    params.append(10.**c)

weights = np.array(weights)
plt.plot(params, weights[:, 0],
         label='petal length')
plt.plot(params, weights[:, 1], linestyle='--',
         label='petal width')
plt.ylabel('weight coefficient')
plt.xlabel('C')
plt.legend(loc='upper left')
plt.xscale('log')
plt.show()

Diskussion
* bei kleinen Werten von C, d.h. bei starker Regularisierung schrumpfen die Gewichtungskoeffizienten 
* Beachte: Für die Regularisierung ist die Standardisierung der Merkmale auch von großer Bedeutung 
    * Wenn die Merkmale zu verschiedene Größenordnungen haben, kann das Probleme machen! 



# Klassifizierung mit Support Vector Machines

Bisher:
* Klassifizierung hat immer auf linearen Entscheidungsgrenzen beruht
* Nach der Optimierung der Gewichte durch Training wurden die Entscheidungsgrenzen visualisiert 



Frage:
* Was ist eine gute Entscheidungsgrenze? 

Ansatz: 
* um die Klassengrenzen ein möglichst breiter (großer) Rand etablieren 
    * Abstand zwischen den trennenden Hyperebenen und den Objekten der Trainingsdatenmenge maximieren 
    * die Objekte bilden dabei Stützvektoren (Support Vector)



![IdeeSVM](./Bilder/BuchKap03/03_09.png) 



Betrachte die parallel zur Entscheidungsgrenze verlaufenden Hyperebenen
* Entscheidungsgrenze: $w_0 + w^T x = 0$
* positive Hyperebene: $w_0 + w^T x_{pos} = 1$
* negative Hyperebene: $w_0 + w^T x_{neg} = -1$

positive Hyperebene - negativer Hyperebene:  

$$w^T (x_{pos} - x_{neg}) = 2$$

Normierung

$$||w|| = \sum_{j=1}^{m} w_j^2$$

$$\Rightarrow \frac{w^T (x_{pos} - x_{neg})}{||w||} = \frac{2}{||w||}$$ 



* linke Seite: normierter Abstand zwischen der positiven und negativen Hyperebene
    * **Margin**: soll maximiert werden 

* rechte Seite: um den **Margin** zu maximieren, muss $\frac{2}{||w||}$ maximiert werden 
    * Mit der Einschränkung, dass die fraglichen Objekte korrekt klassifiziert werden 
        * für alle Objekte i:  $i = 1, ..., N$ 
        * $x_0 + w^T x^{(i)} \ge 1$, wenn $y^{(i)} = 1$
        * $x_0 + w^T x^{(i)} < -1$, wenn $y^{(i)} = -1$

* Alle zur negativen Klasse gehörenden Objekte sind jenseits der negativen Hyperebene
* Alle zur positiven Klasse gehörenden Objekte sind jenseits der positiven Hyperebene


* Einfacher formuliert: 

 $$y^{(i)}(x_0 + w^T x^{(i)}) \ge 1 ~ ~~~~ \forall i$$  

in der Praxis: 
* es ist einfacher den Reziprokenterm zu minimieren: 

$$\frac{||w||^2}{2}$$



### Vorgehen bei nicht linear trennbaren Fällen

**Schlupfvariable**
* Vergleiche Regularisierung
* lockert bei nicht linear trennbaren Fällen die linearen Beschränkungen
* ermöglicht die Konvergenz der Optimierung beim Vorhandensein von Fehlklassifikationen 
* für alle Objekte i:  $i = 1, ..., N$
    * $x_0 + w^T x^{(i)} \ge 1 + \xi^{(i)}$, wenn $y^{(i)} = 1$
    * $x_0 + w^T x^{(i)} < -1 + \xi^{(i)}$, wenn $y^{(i)} = -1$



Änderung der Funktion zur Minimierung: 

$$
\frac{||w||^2}{2} + C \left( \sum_i \xi^{(i)}\right)
$$

Variable C: 
* Steuerung der Bestrafung für Fehlklassifikationen 
    * hohe Werte für C: umfassendere Straffunktion 
    * kleine Werte für C: Fehlklassifizierungen werden weniger streng gehandhabt
* mit C kann 
    * die Margin-Breite bei Fehlklassifizierungen gesteuert werden 
    * ein Bias-Varianz-Kompromiss gefunden werden



![SVMFehlklassifikationen](./Bilder/BuchKap03/03_10.png))



### SVM mit scikit-learn 

Anwendung auf Iris Datensammlung

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', C=1.0, random_state=1)
svm.fit(X_train_std, y_train)

plot_decision_regions(X_combined_std, 
                      y_combined,
                      classifier=svm, 
                      test_idx=range(105, 150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

**Vergleich logistische Regression und SVM**

* Beide liefern sehr ähnliche Ergebnisse
* logistische Regression ist anfälliger für Ausreißer in den Daten 
* SVM sind die Stützvektoren in der Nähe der Entscheidungsgrenzen ausschlaggebend
* logistische Regression ist das einfachere Modell
* logistische Regression ist einfacher zu aktualisieren
    * Vorteile bei Datenströmen



### alternative Implementierungen in scikit-learn 

bisher: 
* Verwendung der Klassen Perceptron und LogisticRegression
    * basieren auf LIBLINEAR 
        * hochoptimierte C/C++ Bibliothek
* Verwendung der Klasse SVC
    * basiert auf LIBSVM
        * hochoptimierte C/C++ Bibliothek speziell für SVM
* Die Verwendung der Bibliotheken beschleunigt die Verarbeitung 
    * Klassifizierer lassen sich schnell trainieren 



Aber: 
* Datenmengen können sehr groß werden 
    * Speicher reicht für schnelle Verarbeitung nicht mehr aus
    * partial_fit wird benötigt

Lösung: 
* Alternative Implementierung: SGDClassifier Klasse
    * unterstützt über partial_fit auch Online-Lernen
    * Implementierung ähnelt dem besprochen Gradientenabstiegsverfahren
    * einfache Implementierung von Perzeptron, logRegression und SVN mit Standardparametern

In [ ]:
from sklearn.linear_model import SGDClassifier

ppn = SGDClassifier(loss='perceptron', max_iter=1000)
lr = SGDClassifier(loss='log', max_iter=1000)
svm = SGDClassifier(loss='hinge', max_iter=1000)

## Nichtlineare Aufgabe für SVN 

Bisher: 
* Klassen waren durch lineare Entscheidungsgrenzen trennbar 



Jetzt: 
* die folgende Datensammlung 
* nicht durch lineare Entscheidungsgrenze trennbar

In [ ]:
np.random.seed(1)
X_xor = np.random.randn(200, 2)
y_xor = np.logical_xor(X_xor[:, 0] > 0,
                       X_xor[:, 1] > 0)
y_xor = np.where(y_xor, 1, -1)

plt.scatter(X_xor[y_xor == 1, 0],
            X_xor[y_xor == 1, 1],
            c='b', marker='x',
            label='1')
plt.scatter(X_xor[y_xor == -1, 0],
            X_xor[y_xor == -1, 1],
            c='r',
            marker='s',
            label='-1')

plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.legend(loc='best')
plt.tight_layout()
plt.show()

* weder logistische Regression noch die bisherige SVM Version könnten das lösen 
* beachte den Aufruf für SVM: 
    * SVC(kernel='linear', C=1.0, random_state=1)



### Lösungsansatz 

* ```kernel``` Parameter kann verändert werden 

Idee: 

<img  src="./Bilder/BuchKap03/03_13.png" style="height:50%"/>




* Transformation $\phi$ in einen höherdimensionierten Raum 

$$\phi(x1, x2) = (z1, z2, z3) = (x_1, x_2, x_1^2 + x_2^2)$$

* Finden einer linearen Hyperebene im höherdimensionierten Raum 

* Rücktransformation $\phi^{-1}$ der Hyperebene



### Kerneltrick 

Ansatz: 
* unbekannte Daten werden transferiert und im neuen Merkmalsraum klassifiziert 

Problem: 
* bei hochdimensionalen Daten kann die Erstellung der neuen Merkmale rechenintensiv sein 

Kerneltrick
* keine Herleitung, grobe Beschreibung
* in der Praxis Ersetzung von Skalarprodukten <br>
${x^{(i)}}^T x^{(j)} = \phi(x^{(i)})^T \phi(x^{(j)})$
* Definition einer Kernelfunktion:  <br>
$k(x^{(i)}, x^{(j)}) = \phi(x^{(i)})^T \phi(x^{(j)})$


* RBF-Kernel (Radial Base Funktion)
    * Auch Gaußscher Kernel
    * häufig eingesetzt

$$k(x^{(i)}, x^{(j)}) = \exp \left( - \frac{\Vert x^{(i)} - x^{(j)}\Vert^2}{2 \sigma^2} \right) \\[4mm]
= \exp \left( - \gamma \Vert x^{(i)} - x^{(j)}\Vert^2 \right) \text{ mit } \gamma = 1/(2\sigma^2)$$



* Beachte: 
    * $\gamma$ ist auch ein freier Parameter (muss clever gewählt werden)
    * Kernel-Funktion entspricht einer Ähnlichkeitsfunktion 
        * exakt gleiche Werte: 1
        * bei sehr unterschiedlichen Werten: 0 

Implementierung
* verwende anderen Kernel (hier rbf)

In [ ]:
svm = SVC(kernel='rbf', random_state=1, gamma=0.10, C=10.0)
svm.fit(X_xor, y_xor)
plot_decision_regions(X_xor, y_xor,
                      classifier=svm)

plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

### Kernel auf Iris-Daten 
* Variiere $\gamma$!!

* mit kleinem $\gamma$

In [ ]:
svm = SVC(kernel='rbf', random_state=1, gamma=0.2, C=1.0)
svm.fit(X_train_std, y_train)

plot_decision_regions(X_combined_std, y_combined,
                      classifier=svm, test_idx=range(105, 150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

### Kernel auf Iris-Daten (II)
* Variiere $\gamma$!!

* mit großem $\gamma$

In [ ]:
svm = SVC(kernel='rbf', random_state=1, gamma=100, C=1.0)
svm.fit(X_train_std, y_train)

plot_decision_regions(X_combined_std, y_combined,
                      classifier=svm, test_idx=range(105, 150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

Beachte:
* das Modell ist gut an Trainingdaten angepasst
* aber bei unbekannten Daten keine Verallgemeinerung und vermutlich viele Fehler
* Optimierung von $\gamma$ wird auch eine wichtige Rolle spielen



# Entscheidungsbäume trainieren 

* attraktive Modelle zur Klassifizierung
    * aus dem täglichen Leben bekannt 
    * basiert auf Fragen (wenn dann...)

<img  src="./Bilder/BuchKap03/03_17.png" style="height:50%"/>



* nur Entscheidungsfragen
    * kategoriale Merkmale

* wie gehen wir mit reellen Zahlen (z.B. Blattbreite bei Iris) um? 
    * in Entscheidungsfragen umwandeln: 
        * z.B. (Blattebreite <= 2.8cm )  = JA/NEIN

* damit haben wir auch schon Parameter um den Baum zu optimieren



**Wie trainineren wir den Baum?** 

* Wurzel des Baumes beinhaltet alle Daten 
* Daten werden so aufgeteilt und der größte Informationsgewinn entsteht
    * $IG$ = Informationsgewinn (Information Gain) 
* bei jedem Kindknoten wiederholen wir diese Aufteilungsprozedur bis zu einem Blattknoten
    * die Elemente an einem Blattknoten gehören dann zu einer Klasse

<br><br><br>

* __Achtung__
    * wenn der Baum beliebig tief werden kann, besteht die Gefahr der Überanpassung
        * im Extremfall hat jedes Trainingselement einen eigenen Blattknoten! 
    * Ansatz: Die Tiefe des Entscheidungsbaums begrenzen! 



## Maximierung des Informationsgewinns

Aufteilung der Daten an einem Knoten
* Für die optimale Aufteilung 
    * Zielfunktion: $IG$ maximieren (_Information Gain_)
    * Variablen:
        * $D_p$: Datenmenge des Elternknotens
        * $N_p$: Anzahl der Exemplare im Elternknoten
        * $D_j$: Datenmenge des $j$-ten Kindknotens
        * $N_j$: Anzahl der Exemplare im $j$-ten Kindknoten
        * $f$: Merkmal der Daten, an dem die Aufteilung stattfinden soll (vgl. $x^{(i)}$)
        * $I$: Maß für die Unreinheit (Impurity)
    * Definition:

$$IG(D_p, f) = I(D_p) - \sum_{j=1}^{m} \frac{N_j}{N_p} I(D_j)
$$

* Unreinheit des Eltern-Knotens minus Summe der Unreinheiten der Kindknoten
    * je kleiner die Unreinheit der Kindknoten, desto größer der Informationsgewinn



Pragmatischer Ansatz: 
* die meisten Bibliotheken verwenden binäre Entscheidungsbäume
* Vereinfachung: 

$$IG(D_p, f) = I(D_p) - \frac{N_{links}}{N_p} I(D_{links}) - \frac{N_{rechts}}{N_p} I(D_{rechts})
$$


Frage: 
* Wie bestimmen wir die Impurity $i$ (Unreinheit)?
* Wir definieren drei weitere beitragende Größen: 
    * Entropie ($I_H$)
    * Gini-Koeffizient ($I_G$)
    * Klassifizierungsfehler ($I_E$)



**Entropie** $I_H$:
* Betrachte einen **Knoten $t$**, der Elemente mehrerer Klassen hat
* $p(i|t)$ Anteil der zur Klasse $i$ gehörenden Exemplare
* Def: Entropie des Knoten $t$ mit $c$ Klasse        
    * mit $p(i|t) \ne 0 ~~~~~ \forall c$ 

$$ I_H(t) = - \sum_{i=i}^{c} p(i|t) \log_2 p(i|t)
$$

* Diskussion: 
    * alle Objekte eine Knoten gehören zu der  Klasse $k$ $\rightarrow$ $p(k|t) = 1$
        * nur ein Beitrag 
        * $I_H = 0$ 
    * Bsp: zwei Klassen gleichverteilt: $\rightarrow$ $p(k|t) = 0.5$ 
        * $I_H(t) = - \sum_{i=i}^{c} 0.5 \log_2 0.5 = - \sum_{i=i}^{c} 0.5 (-1) = 1$ 
    * Entropie ist bestrebt, die wechselseitig im Baum vorhandene Information zu maximieren 



**Gini-Koeffizient** $I_G$: 

* Betrachte einen Knoten $t$, der Elemente mehrerer Klassen $c$ hat
* Maß für die Minimierung der Wahrscheinlichkeit einer Fehlklassifizierung

$$I_G(t) = \sum_{i=1}^{c} p(i|t) (1 - p(i|t)) = 1 - \sum_{i=1}^{c} p(i|t)^2
$$

* Diskussion: 
    * alle gehören zu einer Klasse: $\rightarrow$ $p(k|t) = 0$
    * wenn zwei Klassen ($c=2$) perfekt durchmischt sind: $\rightarrow$ $p(k|t) = 1 - \sum^2 0.5^2 = 0.5$
        * Maximum 
    * $I_H$ und $I_G$ liefern sehr ähnliche Ergebnisse
        * Praxis: Es lohnt sich nicht, viel Zeit in die Auswertung von Bäumen mit verschiedenen Unreinheitskriterien zu stecken



**Klassifizierungsfehler** $I_E$: 

* Betrachte einen Knoten $t$, der Elemente mehrerer Klassen hat
* Kriterium zum **Stutzen** des Entscheidungsbaumes
    * Pruning
* kein Kriterium für Konstruktion 
    * wenig empfindlich für Änderungen der Klassenwahrscheinlichkeiten des Knotens

$$I_E(t) = 1 - \max(p(i|t))  
$$



**Vergleich der Impurity-Ansätze**

Experiment: 
* Wurzelknoten mit zwei Klassen 
    * Klasse 1: 40 Exemplare
    * Klasse 2: 40 Exemplare
    * Schreibweise der Verteilung: (40,40)

* zwei Varianten der Aufteilung: 
    * Variante A
        * linker Kindknoten (30 Exemplare Klasse 1 und 10 Exemplare der Klasse 2) 
        * rechter Kindknoten (10 Exemplare der Klasse 1 und 30 Exemplare der Klasse 2)
    * Variante B
        * linker Kindknoten (20 Exemplare Klasse 1 und 40 Exemplare der Klasse 2) 
        * rechter Kindknoten (20 Exemplare der Klasse 1 und 0 Exemplare der Klasse 2)



![ImpurityExperiment](./Bilder/BuchKap03/03_18.png)



Berechnen des Informationsgewinns mit den drei Ansätzen: 

* Erinnerung

$$IG(D_p, f) = I(D_p) - \frac{N_{links}}{N_p} I(D_{links}) - \frac{N_{rechts}}{N_p} I(D_{rechts})
$$



**Klassifizierungsfehler** 

Fall A
* Wurzelknoten: $I_E(D_p) = 1 - 0.5 = 0.5$
* Linker Knoten: $I_E(D_{links}) = 1 - 3/4 = 0.25$
* Rechter Knoten: $I_E(D_{rechts}) = 1 - 3/4 = 0.25$
* $IG = 0.5 - 4/8 \cdot 0.25 - 4/8 \cdot 0.25 = 0.25$

Fall B
* Wurzelknoten: $I_E(D_p) = 1 - 0.5 = 0.5$
* Linker Knoten: $I_E(D_{links}) = 1 - 4/6 = 1/3 $
* Rechter Knoten: $I_E(D_{rechts}) = 1 - 1 = 0 $
* $IG = 0.5 - 6/8 \cdot 1/3 - 0 = 0.25$

$\rightarrow$ beide Aufteilungen identisch, was den Informationsgewinn betrifft 



**Gini-Koeffizient** 

Fall A
* Wurzelknoten: $I_G(D    _p) = 1 - (0.5^2 + 0.5^2) = 0.5$
* Linker Knoten: $I_G(D_{links}) = 1 - ((3/4)^2 + (1/4)^2) = 0.375$
* Rechter Knoten: $I_G(D_{rechts}) = 1 - ((1/4)^2 + (3/4)^2) = 0.375$
* $IG = 0.5 - 4/8 \cdot 0.375 - 4/8 \cdot 0.375 = 0.125$

Fall B
* Wurzelknoten: $I_G(D    _p) = 1 - (0.5^2 + 0.5^2) = 0.5$
* Linker Knoten: $I_G(D_{links}) = 1 - ((2/6)^2 + (4/6)^2) = 0.444$
* Rechter Knoten: $I_G(D_{rechts}) = 1 - ((1)^2 + (0)^2) = 0 $
* $IG = 0.5 - 6/8 \cdot 0.444 - 0 = 0.166$

Szenario B hat größeren Informationsgewinn $\rightarrow$, B ist "reiner" und damit vorzuziehen



**Entropie**

Fall A
* Wurzelknoten: $I_E(D    _p) = - (0.5 \log_2(0.5) + 0.5 \log_2(0.5)) = 1$
* Linker Knoten: $I_E(D_{links}) = - ((3/4) \log_2(3/4) + (1/4) \log_2(1/4)) = 0.81$
* Rechter Knoten: $I_E(D_{rechts}) = - ((1/4) \log_2(1/4) + (3/4) \log_2(3/4)) = 0.81$
* $IG = 1 - 4/8 \cdot 0.81 - 4/8 \cdot 0.81 = 0.19$

Fall B
* Wurzelknoten: $I_E(D    _p) = - (0.5 \log_2(0.5) + 0.5 \log_2(0.5)) = 1$
* Linker Knoten: $I_E(D_{links}) = - ((2/6) \log_2(2/6) + (4/6) \log_2(4/6)) = 0.92$
* Rechter Knoten: $I_E(D_{rechts}) = - (1 * log_2(1) + [kein Beitrag]) = 0 $
* $IG = 1 - 6/8 \cdot 0.92 - 0 = 0.31$

Szenario B hat größereren Informationsgewinn



#### Darstellung der Impurity-Funktionen

* für eine Klasse 
* im Wertebereich $[0,1]$

Definiere die Impurity-Funktionen

In [ ]:
def gini(p):
    return p * (1 - p) + (1 - p) * (1 - (1 - p))


def entropy(p):
    return - p * np.log2(p) - (1 - p) * np.log2((1 - p))


def error(p):
    return 1 - np.max([p, 1 - p])

Zeichne die Funktionen

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(0.0, 1.0, 0.01)

ent = [entropy(p) if p != 0 else None for p in x]
sc_ent = [e * 0.5 if e else None for e in ent]
err = [error(i) for i in x]

fig = plt.figure()
ax = plt.subplot(111)
for i, lab, ls, c, in zip([ent, sc_ent, gini(x), err], 
                          ['Entropy', 'Entropy (scaled)', 
                           'Gini Impurity', 'Misclassification Error'],
                          ['-', '-', '--', '-.'],
                          ['black', 'lightgray', 'red', 'green', 'cyan']):
    line = ax.plot(x, i, label=lab, linestyle=ls, lw=2, color=c)

ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15),
          ncol=5, fancybox=True, shadow=False)

ax.axhline(y=0.5, linewidth=1, color='k', linestyle='--')
ax.axhline(y=1.0, linewidth=1, color='k', linestyle='--')
plt.ylim([0, 1.1])
plt.xlabel('p(i=1)')
plt.ylabel('Impurity Index')
plt.show()

Diskussion
* Gini und Entropie sehr ähnlich



## Konstruktion eines Entscheidungsbaums

Beachte: 
* Tiefe begrenzen, sonst besteht die Gefahr der Überanpassung
    * hier: Tiefe = 4
* Unreinheitskriterium auswählen 
    * hier: Gini

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='gini', 
                              max_depth=4, 
                              random_state=1)
tree.fit(X_train, y_train)

X_combined = np.vstack((X_train, X_test))
y_combined = np.hstack((y_train, y_test))
plot_decision_regions(X_combined, y_combined, 
                      classifier=tree, test_idx=range(105, 150))

plt.xlabel('petal length [cm]')
plt.ylabel('petal width [cm]')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

Diskussion: 
* Entscheidungsgrenzen des Entscheidungsbaumes laufen entlang der Merkmalsachsen


* Entscheidungsbäume erledigen die Klassifizierung der Blumen sehr gut

* Leider gibt es in scikit-learn keine Möglichkeit, den Baum im Nachhinein zu stutzen
    * Untersuchen Sie unterschiedliche Tiefen

* kann man sich als Graph visualisieren lassen



#### Visualisierung des Entscheidungsbaums

Klassischer Entscheidungsbaum 

* Erstellung mit Graphviz möglich
* Installation über conda
    * conda install pydotplus
    * conda install graphviz
    * conda install python-graphviz




Die Graphviz Executables müssen noch in den Pfad
* in dem Beispiel wurde Anaconda unter dem USER_HOME Verzeichnis installiert.

In [ ]:
import os
os.environ["PATH"] += os.pathsep + os.path.expanduser('~') + '\\Anaconda3\\Library\\bin\\Graphviz'

visualisieren mit dem folgenden Skript

In [ ]:
from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz
from IPython.display import Image

dot_data = export_graphviz(tree,
                           filled=True, 
                           rounded=True,
                           class_names=['Setosa', 
                                        'Versicolor',
                                        'Virginica'],
                           feature_names=['petal length', 
                                          'petal width'],
                           out_file=None) 
graph = graph_from_dot_data(dot_data) 
graph.write_png('tree.png') 

Image('tree.png')

## Random-Forest

RandomForest 
* kann man sich als _Ensemble_ von Entscheidungsbäumen vorstellen 

* wurden sehr populär in der Anwendung von ML wegen  
    * guter Klassifizierungsfähigkeit
    * einfacher Handhabung
    * Skalierbarkeit

Idee: 
* mehrere Entscheidungsbäume mit jeweils großer Varianz
* zu einem stabileren Modell kombinieren, 
* das beim Verallgemeinern besser arbeitet und weniger anfällig für Überanpassung ist 



Einschub: "mit und ohne Ersetzung"

* fünf Kugeln in einer Urne (1,2,3,4,5)
* es wird eine Kugel gezogen
    * Wahrscheinlichkeit eine Kugel zu ziehen ist $1/5$

* zwei Varianten: 
    * mit Ersetzung: die Kugel wird wieder der Urne hinzugefügt
        * Wahrscheinlichkeit bleibt konstant
        * eine Zahl kann mehrmals gezogen werden 
        * Stichproben (hier die Zahlen) sind voneinander unabhängig
            * Kovarianz beträgt 0 
    * ohne Ersetzung: die Kugel (Zahl) wird **nicht** mehr der Urne hinzugefügt
        * Wahrscheinlichkeit der verbliebenen Kugel ändert sich auf 1/4



### Algorithmus Random-Forest

1. Wähle eine zufällige Stichprobe der Größe $n$ aus den Trainingdaten aus (mit Ersetzung)

1. Konstruiere anhand der Stichprobe einen Entscheidungsbaum! Für jeden Knoten werden die folgenden Schritte ausgeführt
    1. Wählen Sie zufällig $d$ Merkmale aus (ohne Ersetzung)
    1. Teilen Sie den Knoten an dem Merkmal auf, das hinsichtlich der Zielfunktion am besten geeignet ist 
        * (z.B. Maximierung des Informationsgehalts)

1. Wiederholen Sie die Schritte 1 und 2 k-mal

1. Fassen Sie die Vorhersagen der einzelnen Bäume durch Mehrheitsentscheidung zusammen , um die Klassenbezeichnung zuzuweisen. 

* Zur Mehrheitsentscheidung kommt später noch mehr! 
* Beachten Sie: in Schritt 2 werden nicht alle Merkmale berücksichtigt, sondern nur ein zufälliger Teil (Untermenge) davon 



**Nachteile** 

* Entscheidungsbäume sind leichter zu interpretieren (z.B. durch Graph)



**Vorteile** 

* Auswahl geeigneter Hyperparameter ist unempfindlich
    * das _EnsembleModell_ ist für das Rauschen einzelner Entscheidungsbäume unempfindlich
* $k$ ist der einzige, in der Praxis relevante Hyperparameter
    * großes $k$
        * zuverlässigere Vorhersagen
        * erhöhte erforderliche Rechenleistung zum Training
* Weitere Hyperparameter
    * $d$: Anzahl der Merkmale für die Aufteilung
    * $n$: Größe der anfänglichen Stichprobe
        * hilft beim Bias-Varianz-Dilemma
        * kleines $n$: Bäume werden verschiedenartiger
            * Rauschen erhöht sich und Überanpassung wird verhindert
            * schlechtere Leistung
        * größere $n$: Überanpassung beachten
            * die verschiedenen Entscheidungsbäume werden ähnlicher
            * Anpassung an die Trainingsdaten



Pragmatischer Praxisansatz

* die meisten Implementierungen 
    * setzen $n$ auf die Größe der ursprünglichen Trainingsdaten  
    * setzen $d$ auf die Wurzel der Merkmale $d = \sqrt{m}$
    * setzen $k$ als Parameter

Implementierung mit scikit_learn: 

* es werden $k = 25$ zufällige Entscheidungsbäume als Ensemble verwendet
    * n_jobs = 2: hilft bei der Parallelisierung (Threads auf 2 Cores)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(criterion='gini',
                                n_estimators=25, 
                                random_state=1,
                                n_jobs=2)
forest.fit(X_train, y_train)

plot_decision_regions(X_combined, y_combined, 
                      classifier=forest, test_idx=range(105, 150))

plt.xlabel('petal length [cm]')
plt.ylabel('petal width [cm]')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

## Einschub: Parametrisierte kontra nichtparametrisierte Modelle

**Parametrisierte Modelle** 

* anhand von Trainingsdaten werden Parameter abgeschätzt, um eine Diskriminanzfunktion herzuleiten, die neue Daten klassifizieren kann
    * kein Rückgriff auf Trainingsdaten notwendig
* Bsp: Perzeptron, logistische Regression, lineare SVM

**Nichtparametrisierte Modelle** 

* lassen sich nicht durch einen festen Satz von Parametern beschreiben 
* Anzahl der Parameter nimmt mit den Trainingsdaten zu
* Bsp: Entscheidungsbaum/RandomForest, Kernel SVM



## k-Nearest Neighbors

* Lazy-Learning-Algorithmus
    * Modellbildung erfolgt nicht durch das Lernen aus Trainingsdaten
    * sondern durch "Speichern" der Trainingsdaten 

* ist nichtparametriesiertes Modell
    * Unterklasse: Instanzbasiertes Lernen
        * Speichern der Trainingsdaten 
        * bei KNN sogar ohne Aufwand



### KNN Algorithmus

<img  src="./Bilder/BuchKap03/03_23.png" style="height:50%; float:right"/>

1. Auswahl der Merkmale und Sammeln der Trainingsdaten 
1. Auswahl der $k$ nächsten Nachbarn des zu klassifizierenden Elements/Exemplars
1. Zuweisung der Klassenbezeichung durch Mehrheitsentscheidung




Parameter
* $k$: Beeinflußt die Überanpassung
* Abstandsmaß
    * muss gewählt werden
    * geeignete Metrik notwendig
    * z.B. reellwertige Daten
        * euklidischer Abstand
        * vorher Standardisierung notwendig 
            * jedes Merkmal so gleichmäßig zum Abstand beitragen
    * Minkowski-Metrik
        * p=2: Euklid-Abstand: 
        * p=1: Manhattan-Metrik  

$$ d(x^{(i)}, x^{(j)}) = \sqrt{\sum_{k} {| x_k^{(i)} - x_k^{(j)} |}^p}
$$



In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5, 
                           p=2, 
                           metric='minkowski')
knn.fit(X_train_std, y_train)

plot_decision_regions(X_combined_std, y_combined, 
                      classifier=knn, test_idx=range(105, 150))

plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

**Vorteile**

* Klassifizierer passt sich bei hinzukommenden Daten sofort (und mit wenig Aufwand) an 

**Nachteile**

* Rechenaufwand für Klassifizierung steigt mit der Anzahl der Trainingsexemplare
    * im schlimmsten Fall linear
* es ist nicht möglich Trainingsdaten zu verwerfen
    * damit steigt der Speicherbedarf linear mit der Anzahl der Trainingsexemplare an



**KNN und der Fluch der Dimensionalität**

* Gegeben sein ein fester Trainingsdatensatz
* sollen mehr Merkmale zu Klassifizierung verwendet werden (die Dimensionalität steigt), dann wird der Merkmalsraum immer dünner besetzt

* bei parametriesierbaren Modellen kann die Regularisierung helfen 
* bei KNN oder Entscheidungsbäumen müssen Verfahren zur Merkmalauswahl helfen ($\rightarrow$ folgenden Kapitel) 



# Zusammenfassung

* scikit-learn angewendet

* viele Algorithmen für überwachtes Lernen kennengelernt
    * lineare und nichtlineare Aufgabenstellungen 
    * Perceptron und Adaline
    
* logistische Regression
    * kann Wahrscheinlichkeiten für ein bestimmtes Verfahren vorhersagen 
    * kann Online-Learning mit stochastischem Gradientenabstiegsverfahren realisieren
  
* Support-Vector-Machines (SVM)
    * können leistungsfähige lineare Modelle erstellen
    * greifen jedoch auf viele Parameter zurück, die fein abgestimmt werden müssen
    * können mit dem Kerneltrick auch auf nichtlineare Aufgabenstellungen angewandt werden 



# Zusammenfassung (II)

* Entscheidungsbäume
    * können leichter interpretiert werden

* Random-Forest als Ensemble-Methode 
    * braucht weniger Feinabstimmung der Parameter 
    * muss weniger Augenmerk auf Überanpassung legen 

* k-Nearest-Neighbor kNN 
    * alternativer Ansatz zur Klassifizierung
    * ist für große Datenmengen speicher- und rechenintensiv
    

**Auswahl des richtigen Algorithmus nicht einfach!** 



**ABER**

noch wichtiger als der Algorithmus ist die **Qualität der Trainingsdatenmenge**

* ohne informative und aussagekräftige Merkmale kann kein Algorithmus funktionieren 

Also sind die folgenden Themen genauso wichtig wie die Algorithmen: 

* Vorverarbeitung von Daten 
* geeignete Merkmalauswahl
* evtl. Dimensionsreduktion